# Estudos relacionado ao DataFrame utilizando o Apache Spark.

### Instalação dos componentes relacionado ao Spark

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
import findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
findspark.init('spark-2.4.4-bin-hadoop2.7')

## Importação das Bibliotecas

In [3]:
import pyspark
from pyspark import SparkContext as sc
from pyspark.sql import SparkSession

## Introdução ao DataFrame

### Criando uma sessão com SparkSession

In [4]:
spark1 = SparkSession.builder.appName('Basics').getOrCreate()

### Leitura de um arquivo JSON 

In [5]:
PATH = '/content/drive/MyDrive/Colab Notebooks/Estudos/Spark/base/people.json'
df = spark1.read.json(PATH)
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



### Esquema dos dados

In [6]:
# Esquema análogo ao SQL, onde pode incluir valores nulos
 
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



### Listagem das colunas do arquivo JSON

In [7]:
df.columns

['age', 'name']

### Descrição Matemática do conteúdo

In [8]:
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



In [9]:
df.summary().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    25%|                19|   null|
|    50%|                19|   null|
|    75%|                30|   null|
|    max|                30|Michael|
+-------+------------------+-------+



### Métodos Take e Collect

In [10]:
df.take(2)

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

In [11]:
df.collect()

[Row(age=None, name='Michael'),
 Row(age=30, name='Andy'),
 Row(age=19, name='Justin')]